We need to access OpenAI API. I use the older version, but there is a new one (1.xx), which is functionally equivalent for these purposes but with a different syntax. Feel free to modify the prompt function if you want to use the latest openai package.

In [112]:
%pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\yeedrag\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


We'll use pandas to have data in a nice data structure, and time to keep track of created files

In [113]:
import pandas as pd
import openai
import os
from time import strftime

This is a simple function to help send and receive data from ChatGPT API. It loops up to 5 times in case OpenAI fails (which happens randomly sometimes). Here you'd also adjust the model parameters, according to the OpenAI API reference (start here: https://platform.openai.com/docs/api-reference/chat)

In [114]:
def prompt(Q):
  tries = 0
  while tries<5:
    try:
      response = openai.ChatCompletion.create(
        model=mmodel,
        messages=Q,
        temperature=0.5,
        #max_tokens=tkn,
        frequency_penalty=0,
        presence_penalty=0,
        request_timeout=199
      )['choices'][0]['message']['content']
      break
    except Exception as e:
      tries = tries+1
      print(f"An error occurred {tries:d}th time: {e}")
  return(Q,response)

Get time and date for convenient file naming

In [115]:
dtime = strftime("%Y_%m_%d-%H%M%S")

Here you choose the model to use and provide the API key


In [116]:
mmodel = "gpt-3.5-turbo-1106"
with open(r'apikey.txt', 'r') as file:
    apikey = file.read().rstrip()
openai.api_key = apikey

Here we have the prompts we will be using, as well as the system prompt which servers as a general global set of instructions or information. The system prompt may be left empty.

I have two prompts, one asks to generate a table, and the second one iteratively asks to continue that table. This is likely not an optimal approach, but it gives some results so a good example.

I also separated the property from the prompt so that I can vary the property and keep the same prompts, and the othery way around. It does not have to be that way if more task specific prompts work better.

In [117]:
PROPERTY = 'Polymer Transition Temperature'

system = 'You follow instructions carefully, you provide as many rows of the table as possible, at least 50 for every prompt.'

qs = ["Provide me with a list of "+PROPERTY+" values for different materials. Your response should be a table consisting of 2 columns: material, value. The materials have to be typed as unique chemical compositions consisting of chemical element abbreviations and numbers only (e.g. GaAs, but not Gallium Arsenide). The values have to be single numbers, not ranges. Type out as many different values as you can. You are not allowed to type anything else than this table.",
      "Continue expanding this table with new values of "+PROPERTY+" making sure you do not duplicate entries. Type out as many different values as you can. You are not allowed to type anything else than this table."]


In [118]:
prompt_file = r'prompts\rudely_1.txt'
with open(prompt_file, 'r') as file:
    file_content = file.read()
qs = eval(file_content)
print(qs)

['As a materials scientist researcher, provide me with a concise table of Polymer Transition Temperature values. Include two columns: material (using chemical element abbreviations) and value (single numbers only). Type out as many different values as possible. You are not allowed to include anything else.', 'As a materials scientist researcher, continue expanding this table with new values of Polymer Transition Temperature. Avoid duplicates. You are not allowed to type anything else than this table.\n---\nMaterial | Value\nGaAs | 1230\nPEEK | 343\nSiC | 900\nAl2O3 | 1800']


Give the file a uniquely identifiable name. We want to save EVERYTHING or we will lose it, and each generation costs money. Either link up your google drive, or download all files each time because they are temporary (if executed on colab).

In [119]:
filename = PROPERTY.replace(" ", "")+'_'+os.path.basename(prompt_file)[:-4]+'_'+dtime+'.csv'
print(filename)
print(f"Saving to: {filename} and {filename.replace('csv','txt')}")

PolymerTransitionTemperature_rudely_1_2024_03_12-153934.csv
Saving to: PolymerTransitionTemperature_rudely_1_2024_03_12-153934.csv and PolymerTransitionTemperature_rudely_1_2024_03_12-153934.txt


Here we set up a structure for the chat. I like to hold the conversation in a list, and then each prompt and response inside it are dictionaries, as per OpenAI requirements. I start with just the system prompt and will append to that later.

I also set up some empty lists and initial values to keep track of progress

In [120]:
sss = [{"role": "system", "content": system}]
tab = []
tab_clean = []
ur = 0
um = 0
i = 0

This is the main loop that will loop over the prompts, get responses and try to put them in a structured data format.

This while approach is VERY SIMPLE and does not account for many things that may be the response deom the model, so it is only an example to build upon based on the result, not a solution ready to be used. It does not even have to be used at all if one has a better/different idea.

In [121]:
while True:
  # here i send my first prompt first and then loop over the second one over and
  # over again. You will likely have a different approach to this.
  if i<1:
    qq = qs[0]
  else:
    qq = qs[1]
  # save the first prompt to the conversation
  sss.append({"role": "user", "content": qq})
  # send out the first prompt and receive the response
  sss,ans = prompt(sss)
  # save the response to the conversation
  sss.append({"role": "assistant", "content": ans})

  # we are saving the raw prompts and raw responses, in case we want to analyze
  # or postprocess later
  with open(filename.replace('csv','txt'), 'a') as file:
    print("USER: "+qq, file=file)
    print("GPT : "+ans, file=file)

  # Here we start to grab the data into a nicer structure. For simplicity I had
  # a lot of assumptions. I assume that the word 'value' exists in the header
  # (see first prompt), I remove the header, the separator, and get the rest.
  lines = ans.split('\n')
  if 'value' in lines[0].lower():
    ans = '\n'.join(lines[1:])
  lines = ans.split('\n')
  if '----' in lines[0].lower():
    ans = '\n'.join(lines[1:])
  lines = ans.split('\n')

  # here I try to split the table by columns. I'm assuming that they are
  # separated with |, which is not necessary always the case.
  tab.append(ans)
  try:
    for line in tab[-1].strip().split('\n'):
      tab_clean.append(line.strip('|').split('|'))
  except:
    pass

  # another assumption - only two columns, "material" and "value" (see prompt)
  # some cleanup, converting strings to numbers etc.
  # there is no error handling or edge cases, for example 1.6-1.7 will be removed
  # because it technically is not a number (although it kind of is).
  df = pd.DataFrame(tab_clean, columns=['Material', 'Value'])
  df = df[pd.to_numeric(df['Value'], errors='coerce').notna()]
  df['Value'] = pd.to_numeric(df['Value'])
  df['Material'] = df['Material'].str.strip()
  df.to_csv(filename, index=False)

  # here I count how many new (non-duplicate) materials we are extracting each
  # time, to monitor progress. We stop if more than 10 iterations or not progress
  if len(df.drop_duplicates()) > ur or df['Material'].nunique() > um:
    ur = len(df.drop_duplicates())
    um = df['Material'].nunique()
    i = i+1
    if i > 10:
      print("Stopping due to 10 iterations exceeded")
      break
  else:
    print("Stopping due to NO PROGRESS")
    break

  print(f"Iteration: {i:3} Generated_rows: {len(lines):3};     TOTAL:  Uniq_rows: {len(df.drop_duplicates()):4d}   Uniq_materials: {df['Material'].nunique():4d}")

print(df)

Iteration:   1 Generated_rows:  49;     TOTAL:  Uniq_rows:   33   Uniq_materials:   33
Iteration:   2 Generated_rows:  36;     TOTAL:  Uniq_rows:   36   Uniq_materials:   36
Stopping due to NO PROGRESS
   Material  Value
0        PS    100
1       PVC     87
2      PMMA    105
3        PE     80
4        PP    130
..      ...    ...
80      PFA    305
81      MFA    200
82     GaAs   1230
83      SiC    900
84    Al2O3   1800

[85 rows x 2 columns]
